# Tracking the NASA Satellite
We will be working on streaming data which is satellite location coordinates being provided by NASA.

The data is in the following format:
{ "message": "success", "iss_position": {"longitude": "-56.8155", "latitude": "-42.5979"},"timestamp": 1667492679}

Data can be accessed from [ISS Live tracking](http://api.open-notify.org/iss-now.json)

For the live streaming, we consider the following - 

- The link can be thought of as the producer as it will do producer job.
- The consumer connects and fetches data. The consumer takes the data from producer and uses the location coordinates to plot the satellite location on the world map.
- The streaming interval will be 5 seconds.
- The producer will run for an hour.
- The graph will show the satellite tracking for 1 hr.
- The map with the marking of the locations of the Space Station at every 5s will be saved as an HTML file.

In [1]:
import requests
import folium
import time

## Consumer Job

The Consumer job is defined by the following three functions.

`fetch_satellite_data()` function utilizes the streaming data source or the [Producer URL](http://api.open-notify.org/iss-now.json) for continuous data extraction.

Further, the extracted data is returned to the main function for visualization on world map.

In [2]:
# Function to fetch satellite data from the API
def fetch_satellite_data():
    url = "http://api.open-notify.org/iss-now.json" #The producer
    response = requests.get(url) #the requests library connects to the url
    
    if response.status_code == 200: #checks whether the connection is a success
        data = response.json() #if the connection to the url is a success, the json data at producer is stored in data and returned
        return data
    else:
        print("Failed to fetch data from the API.")
        return None

### Mapping the Locations

We define two functions which gives us the desired output.

- `display_satellite_on_map()`: This function is the heart of the Consumer Job. It processes the extracted data to segregate the location coordinates and timestamp for mapping
- `update_map_with_satellite_location()`: This function ensures that the map is updated with a marker after each 5 seconds denoting the ISS location at that instant

In [3]:
# Function to update the map with satellite location data
def update_map_with_satellite_location(latitude, longitude, timestamp, map_object):
    #we define a Folium Marker object with latitude and longitude as the coordinates and the timestamp pops up on clicking each marker
    marker = folium.Marker([float(latitude), float(longitude)], popup=("Timestamp: ",timestamp))
    #The Marker object is then used to map the location coordinate on a map object which is defined by the other function
    marker.add_to(map_object)

`display_satellite_location_on_map` function does the following -

- Creates a void world map object `satellite_map` using the Folium.Map() with (0,0) as origin coordinates and a zoom level of 2
- The time period for the observation is defined using the `time` module. We require streaming for 1 hour or 3600 seconds
- The function is run as long as the time period is within 3600s implemented using a `while` condition
- For the `TRUE` values of `satellite_data` as extracted by calling the `fetch_satellite_data()` function upon satisfying the condition, `latitude` and `longitude` coordinates are extracted separately and passed to `update_map_with_satellite_location()` function along with the `satellite_map` which is updated at every 5 seconds with the new location marker.
- The `time.sleep()` function implements the 5 seconds gap in streaming.
- Finally, the updated `satellite_track` is saved as an HTML file refreshed every 5 seconds of execution

In [4]:
# Function to display satellite location on a world map
def display_satellite_location_on_map(time_limit):
    # Initialize the world map
    map_center = [0, 0]  # Center coordinates
    satellite_map = folium.Map(location=map_center, zoom_start=2) 
    start_time = time.time()
    end_time = start_time + time_limit
    # Fetch satellite data and update the map every 5 seconds
    while time.time() < end_time:
        satellite_data = fetch_satellite_data() #extracts and returns the JSON data 
        if satellite_data:
            latitude = satellite_data["iss_position"]["latitude"]
            longitude = satellite_data["iss_position"]["longitude"]
            timestamp = satellite_data["timestamp"] #Gives the timestamp for pop up feature on click
            update_map_with_satellite_location(latitude, longitude, timestamp, satellite_map) #the world map is updated with a new marker each time
            time.sleep(5)  # Fetch data every 5 seconds
        else:
            time.sleep(1)  # Wait for 1 second before retrying in case of API failure

        # Save the updated map to an HTML file
        satellite_map.save("satellite_track.html")

## Execution of the Mapping

The __main__ function is called for executing the program and runs for an hour

In [5]:
if __name__ == "__main__":
    time_limit_sec = 3600
    display_satellite_location_on_map(time_limit_sec)#The time limit ensures the execution stops after the defined time.
#For applications that require continuous streaming, the time_limit parameter can be removed from the display_satellite_location_on_map()